In [4]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)

# Access the Feature Store
fs = project.get_feature_store()

# Specify the feature group and its version
feature_group_name = "final_df_feature_group"
feature_group_version = 1

try:
    # Retrieve the 'final_df_feature_group' feature group by its name and version
    final_df_fg = fs.get_feature_group(feature_group_name, version=feature_group_version)
    
    # Read the feature group as a Pandas DataFrame
    final_df = final_df_fg.read()

    # Print success message and display the first few rows of the DataFrame
    print(f"Downloaded feature group: {feature_group_name} (version {feature_group_version})")
    print(final_df.head())  # Optionally display the first few rows of the DataFrame

except RestAPIError as e:
    print(f"Error downloading feature group: {feature_group_name} (version {feature_group_version})")
    print(e)

# Now you have the `final_df` DataFrame containing data from 'final_df_feature_group'


Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.75s) 
Downloaded feature group: final_df_feature_group (version 1)
   unique_id  truck_id    route_id            departure_date  \
0       2172  31794890  R-d1a6a43c 2019-01-16 06:00:00+00:00   
1       5331  54036321  R-11ea3024 2019-01-01 06:00:00+00:00   
2       3135  32048264  R-6cb722d8 2019-01-21 06:00:00+00:00   
3       5129  20339561  R-3d1310d5 2019-01-04 06:00:00+00:00   
4       2938  53208128  R-ddae5148 2019-01-10 06:00:00+00:00   

          estimated_arrival  route_avg_temp  route_avg_wind_speed  \
0 2019-01-17 06:00:00+00:00       62.800000              9.600000   
1 2019-01-02 18:00:00+00:00       49.166667             10.166667   
2 2019-01-22 18:00:00+00:00       61.142857              4.571429   
3 2019-01-05 00:00:00+00:00       69.500000      

In [6]:
final_df.shape

(10290, 49)

In [7]:
final_df.isnull().sum()

unique_id                         0
truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_precip                  0
route_avg_humidity                0
route_avg_visibility              0
route_avg_pressure                0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
temp_origin                       0
wind_speed_origin                 0
description_origin                0
precip_origin                     0
humidity_origin                   0
visibility_origin                 0
pressure_origin                   0
temp_destination                  0
wind_speed_destination            0
description_destination     

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

class Phase2Pipeline:
    
    cts_cols = ['route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip', 'route_avg_humidity', 
                    'route_avg_visibility', 'route_avg_pressure', 'distance', 'average_hours',
                    'temp_origin', 'wind_speed_origin', 'precip_origin', 'humidity_origin', 
                    'visibility_origin', 'pressure_origin', 'temp_destination', 'wind_speed_destination',
                    'precip_destination', 'humidity_destination', 'visibility_destination',
                    'pressure_destination', 'avg_no_of_vehicles', 'truck_age', 'load_capacity_pounds', 
                    'mileage_mpg', 'age', 'experience', 'average_speed_mph']

    cat_cols = ['route_description', 'description_origin', 'description_destination', 'accident', 
                    'fuel_type', 'gender', 'driving_style', 'ratings', 'is_midnight']

    target = ['delay']
    
    
    def __init__(self, feature_store, feature_group_name):
        self.feature_store = feature_store
        self.feature_group_name = feature_group_name

    def load_final_df(self):
        # Load final DataFrame from feature store
        fg = self.feature_store.get_feature_group(name=self.feature_group_name, version=1)
        final_df = fg.read()
        return final_df

    def handle_null_values(self, df):
        # Fill null values in continuous columns with the median
        

        # Fill nulls in continuous columns
       df[cts_cols] = df[cts_cols].fillna(df[cts_cols].median())

        # Fill nulls in categorical columns
       df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

       return df

    def train_validation_test_split(self, df):
        # Remove ID columns if any
        df = df.drop(columns=['unique_id', 'truck_id', 'route_id'])

        # Split features and target
        X = df[cts_cols + cat_cols]
        y = df[target]

        # Split data into train, validation, and test sets
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        return X_train, X_val, X_test, y_train, y_val, y_test

    def execute_pipeline(self):
        # Load the final DataFrame
        final_df = self.load_final_df()

        # Handle null values
        final_df_clean = self.handle_null_values(final_df)

        # Split the data
        X_train, X_val, X_test, y_train, y_val, y_test = self.train_validation_test_split(final_df_clean)

        return X_train, X_val, X_test, y_train, y_val, y_test

# Example usage
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)
fs = project.get_feature_store()

pipeline = Phase2Pipeline(fs, "final_df_feature_group")
X_train, X_val, X_test, y_train, y_val, y_test = pipeline.execute_pipeline()

# You can now proceed with the training, validation, and testing.


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.80s) 


KeyError: "['origin_temp', 'origin_wind_speed', 'origin_precip', 'origin_humidity', 'origin_visibility', 'origin_pressure', 'destination_temp', 'destination_wind_speed', 'destination_precip', 'destination_humidity', 'destination_visibility', 'destination_pressure'] not in index"